In [1]:
import numpy as np
from matplotlib import pyplot as plt
np.random.seed(42)

In [2]:
# different broadcast rules in numpy
shapes = [[(3,1),(3,4)],[(3,1),(1,4)],[(3,1),(3,1)],[(3,4),(3,1)],[(3,4),(1,4)],[(3,4),(3,4)]]
for aa,bb in shapes:
    a = np.random.randn(*aa)
    b = np.random.randn(*bb)
    print(a.shape,'[+-*/]',b.shape,'=',(a-b).shape)

(3, 1) [+-*/] (3, 4) = (3, 4)
(3, 1) [+-*/] (1, 4) = (3, 4)
(3, 1) [+-*/] (3, 1) = (3, 1)
(3, 4) [+-*/] (3, 1) = (3, 4)
(3, 4) [+-*/] (1, 4) = (3, 4)
(3, 4) [+-*/] (3, 4) = (3, 4)


In [98]:
w1 = np.random.rand(100).reshape(1,-1)
w2 = np.random.rand(100).reshape(-1,1)
np.dot(w1,w2)
w1.shape,w2.shape

((1, 100), (100, 1))

In [17]:
# build a simple logistic regression from scratch and use gradient descent

class LR:
    def __init__(self,inp_size=2) -> None:
        # z = x1 * w1 + x2 * w2 + b
        self.w = np.random.rand(inp_size,1) #  left features X right features[<>]
        self.b = np.random.rand(1,1)
    
    def fpass(self,x):
        z = x @ self.w + self.b
        a = self.sigmoid(z)
        return a
    
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))

    def BCE(self,target,predicted):
        return -(target*np.log(predicted) + (1-target)*np.log(1-predicted))
    
    def train(self,x,y,epoch=100,lr=0.01):
        batch = x.shape[0]
        for _ in range(1,epoch+1):
            if _%10==0:
                print(self.w,self.b)
            z = x @ self.w + self.b # (z = xw + b)
            a = self.sigmoid(z)
            loss = self.BCE(y,a)
         
            da = -(y/a - (1-y)/(1-a))
            dz = da * (a - a**2) #dz = a - y
            dw = (x.T @ dz)/batch
            db = dz.sum(axis=0)/batch

            self.w -= lr*dw
            self.b -= lr*db





In [18]:
np.random.seed(42)
model = LR(2)
x = np.random.rand(64,2)
y = np.random.rand(64,1)
model.train(x,y)

[[0.36002527]
 [0.93590538]] [[0.70417022]]
[[0.34425671]
 [0.91976212]] [[0.67396657]]
[[0.32886769]
 [0.90394858]] [[0.64451325]]
[[0.31385917]
 [0.88846648]] [[0.6158102]]
[[0.29923137]
 [0.87331686]] [[0.58785601]]
[[0.28498377]
 [0.85850015]] [[0.56064798]]
[[0.27111518]
 [0.84401614]] [[0.53418209]]
[[0.25762372]
 [0.82986402]] [[0.50845315]]
[[0.24450688]
 [0.81604238]] [[0.4834548]]
[[0.23176154]
 [0.80254927]] [[0.4591796]]
